In [28]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error

import numpy as np
import pandas as pd
import json

In [9]:
with open('encoded_job_onehot.json','r') as f:
    data_dict = json.load(f)
    

In [15]:
def get_inputs(data_keys,data_dict):

    target = [data_dict[k]['Cardinality'] for k in data_keys]

    all_tables = [data_dict[k]['Tables'] for k in data_keys]
    tables_input = tf.keras.preprocessing.sequence.pad_sequences(all_tables,padding='post',dtype='float64')
    tables_input = np.array(tables_input).tolist()
#     print(len(tables_input))

    all_joins = [data_dict[k]['Joins'] for k in data_keys]
    joins_input = tf.keras.preprocessing.sequence.pad_sequences(all_joins,padding='post',dtype='float64')
    joins_input = np.array(joins_input).tolist()
#     print(len(joins_input))

    all_ands = [data_dict[k]['Predicate String'] for k in data_keys]
    and_input = tf.keras.preprocessing.sequence.pad_sequences(all_ands,padding='post',dtype='float64')
    and_input = np.array(and_input).tolist()
#     print(len(and_input))

    all_norm_ands = [data_dict[k]['Predicate Numeric'] for k in data_keys]
    normalized_and_input = tf.keras.preprocessing.sequence.pad_sequences(all_norm_ands,padding='post',dtype='float64')
    normalized_and_input = np.array(normalized_and_input).tolist()
#     print(len(normalized_and_input))
    
    return target,tables_input,joins_input,and_input,normalized_and_input


In [29]:
data_keys = data_dict.keys()
train_target,train_tables_input,train_joins_input,train_and_input,train_normalized_and_input = get_inputs(data_keys,data_dict)


107
107
107
107


In [33]:
df_all = pd.DataFrame()
df_all['Tables'] = train_tables_input
df_all['Joins'] = train_joins_input
df_all['Predcate String'] = train_and_input
df_all['Predcate Numeric'] = train_normalized_and_input
df_all['Cardinality'] = train_target
df_all.reset_index(drop=True, inplace=True)
# print(df_all)
# df_all.to_csv('regressor_train_dataset.csv',index=False)


In [36]:
df_copy = df_all.copy()
# train_set = df_copy.sample(frac=0.75, random_state=0)
train_set = df_copy[:70]
test_set = df_copy.drop(train_set.index)

print ('Training set')
print (train_set)
print ('\nTest set')
print (test_set)

Training set
                                               Tables  \
0   [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...   
1   [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...   
2   [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...   
3   [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...   
4   [[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...   
..                                                ...   
65  [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...   
66  [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...   
67  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...   
68  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...   
69  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...   

                                                Joins  \
0   [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...   
1   [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...   
2   [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...   
3   [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...   
4   [[0.0, 0.0, 0

In [37]:
train_set_labels = train_set.pop('Cardinality')
test_set_labels = test_set.pop('Cardinality')

In [39]:
# print(train_set_labels)

0     0.295331
1     0.028092
2     0.028092
3     0.048023
4     0.573184
        ...   
65    0.518831
66    0.948349
67    0.194225
68    0.148270
69    0.547482
Name: Cardinality, Length: 70, dtype: float64
